Import system dependencies

In [21]:
import sys
import os
sys.path.append('../')
import shutil
import statistics
import numpy as np

Import custom packages

In [22]:
from Utils import shuffleDataset, SplitJPData, saveToPickle, loadFromPickle, SplitChopinData, SplitMozartData, SplitBachData
from JPDataPreProcessing import toVectorTrainFormat, toVectorTestFormat, toInterleavedTrainFormat, toVectorFutureTrainFormat
from TrainModel import trainModel
from TestVectorModel import loadModel, testVecModelSave, testVecModelEval
from EvaluateJPMethod import getScoresForDL, getScoresForHmm

from ReferenceHMM import GetESTFingering
from parameters import DATA_DIR, HMM_RES_DIR

In [23]:
# Change parameters here
composer  = 'bach'

hmm_level = 'FHMM{}'.format(1)

Split Dataset According to Composer

In [24]:
if composer == 'Bach':
    train_files, test_files, hmm_res_files = SplitBachData(DATA_DIR)
elif composer == 'Mozart':
    train_files, test_files, hmm_res_files = SplitMozartData(DATA_DIR)
elif composer == 'Chopin':
    train_files, test_files, hmm_res_files = SplitChopinData(DATA_DIR)
else:
    raise Exception('Wrong Composer')

Train all 3 model of reference HMM with the data splited above

In [25]:
os.chdir('../ReferenceHMM')
GetESTFingering.prepareInputList(train_files)
GetESTFingering.trainHmm()

Run trained HMM model, change GetESTFingering.FHMMx to swith models

In [26]:
txt_dir = os.path.join('ESTResults/selfTrained',   hmm_level)
csv_dir = os.path.join('JPESTResults/selfTrained', hmm_level)

if not os.path.exists(txt_dir):
    os.makedirs(txt_dir)

if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)
    
test_filenames = GetESTFingering.getFormattedTestFilenames(test_files)
GetESTFingering.runHmm(test_filenames, hmm_level, default=False)

GetESTFingering.convertToCsv(
    input_dir  = txt_dir,
    output_dir = csv_dir
)

move the files the to correct folders

In [27]:
if os.path.exists(HMM_RES_DIR + hmm_level):
    shutil.rmtree(HMM_RES_DIR + hmm_level)
shutil.move('JPESTResults/selfTrained/' + hmm_level, HMM_RES_DIR)
os.chdir('../LSTM Approach')

Evaluate hmm model

In [28]:
getScoresForHmm(test_files, hmm_res_files, hmm_level)

Total mean: 
M_GEN:  0.5724991389949875
M_HIGH:  0.6403457066119016
M_SOFT:  0.8447943944896392


Train DeepLearning model

In [35]:
train_input_list, train_label_list = toVectorFutureTrainFormat(train_files, DATA_DIR)
trainModel(train_input_list, train_label_list, num_epochs = 8, batch_size=8)

Epoch 1/8
3929/3929 [==============================] - 59s 14ms/step - loss: 1.1364 - categorical_accuracy: 0.5102

Epoch 00001: saving model to trained_models/cp.ckpt
Epoch 2/8
3929/3929 [==============================] - 58s 15ms/step - loss: 0.9426 - categorical_accuracy: 0.6217

Epoch 00002: saving model to trained_models/cp.ckpt
Epoch 3/8
3929/3929 [==============================] - 58s 15ms/step - loss: 0.7930 - categorical_accuracy: 0.6882

Epoch 00003: saving model to trained_models/cp.ckpt
Epoch 4/8
3929/3929 [==============================] - 58s 15ms/step - loss: 0.6630 - categorical_accuracy: 0.7441

Epoch 00004: saving model to trained_models/cp.ckpt
Epoch 5/8
3929/3929 [==============================] - 59s 15ms/step - loss: 0.5874 - categorical_accuracy: 0.7722

Epoch 00005: saving model to trained_models/cp.ckpt
Epoch 6/8
3929/3929 [==============================] - 59s 15ms/step - loss: 0.5083 - categorical_accuracy: 0.7983

Epoch 00006: saving model to trained_models/

Eval DeepLearning model

In [36]:
model = loadModel()

Evaluation of mulitple fingering for a single piece

In [37]:
getScoresForDL(test_files, hmm_res_files, model)

Total mean: 
M_GEN:  0.49459820647750596
M_HIGH:  0.5382149169361405
M_SOFT:  0.7576591694956264
